In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
x = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                    delimiter = ',', usecols = (4,5,6), skip_header=1)
y = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                   delimiter =',', usecols = (7),skip_header=1)
x_cv = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                    delimiter = ',', usecols = (4,5,6), skip_header=1)
y_cv = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                   delimiter =',', usecols = (7),skip_header=1)
test_dataset = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                              delimiter = ',', usecols = (4,5,6,7), skip_header=1)
train_dataset = np.genfromtxt ('C:/Users/hongy/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                              delimiter = ',', usecols = (4,5,6,7), skip_header=1)

In [4]:
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

In [5]:
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
num_input = 3 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

In [7]:
y_train = np.reshape(y, (54,1)).astype(np.float32)
y_test = np.reshape (y_cv, (14,1)).astype(np.float32)
x_whole = np.vstack ((x, x_cv)).astype(np.float32)
enc = OneHotEncoder ()
x_enc = enc.fit_transform (x_whole).toarray().astype(np.float32)
x_train = x_enc [:54, :]
x_test = x_enc [54:, :]

In [8]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    
    x = x_train
    # Hidden fully connected layer with 20 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 20 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [9]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Build the neural network
    logits = neural_net(features)

    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_spec

In [10]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x, y=y, batch_size=batch_size, num_epochs=10, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

NameError: name 'tf' is not defined